# 1. 0I+3X+5Y+7Z

In [109]:
from math import *
import numpy as np
import scipy.linalg as la
import random 
import cmath 
import math

In [22]:
from qiskit import QuantumRegister, ClassicalRegister
from qiskit import QuantumCircuit, execute, Aer
import pennylane as qml
import numpy as np
from numpy import pi
simulator = Aer.get_backend('qasm_simulator')

from scipy.linalg import expm, sinm, cosm

In [60]:
def y_point_1(x):
    y_temp = []
    a1 =       24.26 
    a2 =      -1.085  
    b1 =       37.21  
    c1 =       7.283  
    c2 =     0.01759 
    d1 =        0.62  
    d2 =      0.7687  
    e =      0.8706  
    for i in x:
        temp = a1/(b1+c1*exp(-d1*i))+a2/(1+c2*exp(-d2*i))+e
        y_temp.append(temp)
    return(y_temp)

In [61]:
def theta_point_1(x):
    z_temp = []
    a1 =      -2.661  
    a2 =       4.206  
    c1 =    0.004604  
    c2 =    0.005635  
    d1 =       1.033  
    d2 =       1.307  
    e =      0.7446  
    for i in x:
        temp = a1/(1+c1*exp(-d1*i))+a2/(1+c2*exp(-d2*i))+e
        z_temp.append(temp)
    return(z_temp)

In [62]:
def measure_z_circuit(given_circuit):
    z_meas = QuantumCircuit(1,1)
    z_meas.append(given_circuit,[0])
    z_meas.measure(0,0)
    return z_meas

def measure_z(given_circuit, num_shots = 10000):

    z_meas = measure_z_circuit(given_circuit)
    
    result = execute(z_meas, backend = simulator, shots = num_shots).result()
    counts = result.get_counts(z_meas)

    if '0' not in counts:
        counts['0'] = 0
    if '1' not in counts:
        counts['1'] = 0

    total_counts = counts['0'] + counts['1']
    z = counts['0'] - counts['1']
    z = z / total_counts
    
    return z

def measure_x_circuit(given_circuit):
    
    x_meas = QuantumCircuit(1,1)
    x_meas.append(given_circuit,[0])
    x_meas.h(0)
    x_meas.measure(0,0)

    return x_meas

def measure_x(given_circuit, num_shots = 10000):
    
    x_meas = measure_x_circuit(given_circuit)
    
    result = execute(x_meas, backend = simulator, shots = num_shots).result()
    counts = result.get_counts(x_meas)

    if '0' not in counts:
        counts['0'] = 0
    if '1' not in counts:
        counts['1'] = 0

    total_counts = counts['0'] + counts['1'] 
    x = counts['0'] - counts['1']
    x = x / total_counts
    
    return x

def measure_y_circuit(given_circuit):
    
    y_meas = QuantumCircuit(1,1)
    y_meas.append(given_circuit,[0])
    y_meas.rz(-pi/2,0)
    y_meas.h(0)

    y_meas.measure(0,0)

    return y_meas

def measure_y(given_circuit, num_shots = 10000):
    
    y_meas = measure_y_circuit(given_circuit)
    
    result = execute(y_meas, backend = simulator, shots = num_shots).result()
    counts = result.get_counts(y_meas)

    if '0' not in counts:
        counts['0'] = 0
    if '1' not in counts:
        counts['1'] = 0

    total_counts = counts['0'] + counts['1'] 
    y = counts['0']  - counts['1']
    y = y / total_counts
    
    return y

In [151]:
X = np.matrix('0,1;1,0')
Y = np.matrix('0,-1j;1j,0')
Z = np.matrix('1,0;0,-1')
I = np.matrix('1,0;0,1')
e = 3
a=1+e
b = 2.3+e
c = 6.2+e
d = 8.8+e
M = a*I+b*X+c*Y+d*Z

In [172]:
def mat(n):
    M_temp = n*I+(3+n)*X+(n+5)*Y+(n+7)*Z
    return(M_temp)

In [153]:
coeffs, obs_list = qml.utils.decompose_hamiltonian(M)
H = qml.Hamiltonian(coeffs, obs_list)

In [66]:
def coef(H):
    coeff = [0,0,0,0]
    h = str(H)
    for i in range (len(h)):
        p = ""
        if h[i] =="(":
            j = i+1 
            while h[j]!=")" :
                p=p+h[j]
                j+=1
            
            if h[j+3] == "I":
                coeff[0] = float(p)
                
            if h[j+3] == "X":
                coeff[1] = float(p)
                
            if h[j+3] == "Y":
                coeff[2] = float(p)
                
            if h[j+3] == "Z":
                coeff[3] = float(p)
    
    return(coeff)

In [149]:
def eigen_value_check(qc,H):
    num_shots = 100000
    z = measure_z(qc, num_shots = num_shots)
    x = measure_x(qc, num_shots = num_shots)
    y = measure_y(qc, num_shots = num_shots)
    coeff =coef(H) 
    ev = coeff[0]+coeff[1]*x+coeff[2]*y+coeff[3]*z
    
    return ev

In [68]:
qc_a = QuantumCircuit(1)
qc_a.initialize([ 0.32296885+0.j,-0.33861069-((1-0.33861069**2-0.32296885**2)**0.5)*1.j],[0])

In [154]:
eigen_value_check(qc_a,H)

-11.788444

In [70]:
results = la.eig(M)

In [71]:
results[0]

array([ 19.87356293-1.13964669e-16j, -11.87356293-7.74213751e-16j])

In [72]:
min(results[0].real)

-11.873562927080991

In [73]:
def Rand(start, end, num): 
    res = [] 

    for j in range(num): 
        res.append(random.randint(start, end)) 

    return res 



In [145]:
num = 100
start = -500
end = 500
x = Rand(start, end, num)

In [146]:
def state(a,b):
    temp = (1-a**2)**0.5
    return([a,temp*cos(b)+temp*sin(b)*1j])
    

In [147]:
y = y_point_1(x)
t = theta_point_1(x)

In [183]:
data_list_1 = [0]*6
error_list_1 = []
exact_1 = []
predicted_1 = []
for i in range(len(x)):
    M = mat(x[i])
    results = la.eig(M)
    eigenval = min(results[0].real)
    
    
    
    qc= QuantumCircuit(1)
    qc.initialize(state(y[i],t[i]),[0])
    coeffs, obs_list = qml.utils.decompose_hamiltonian(M)
    H = qml.Hamiltonian(coeffs, obs_list)
    
    eigenval_predict = eigen_value_check(qc,H)
    
    
    error = (eigenval - eigenval_predict)/(eigenval)
    
    if error <0.05:
        data_list_1[0]+=1
    if 0.1>error :
        data_list_1[1]+=1
    if 0.15>error:
        data_list_1[2]+=1
    if 0.2>error :
        data_list_1[3]+=1
    if 0.25>error:
        data_list_1[4]+=1
    error_list_1.append(error)
    data_list_1[5]+=1
    
    exact_1.append(eigenval)
    predicted_1.append(eigenval_predict)
    

In [187]:
print("Total Test Points", data_list_1[5])
print("Prediction With error less then 5% :- ", data_list_1[0])
print("Prediction With error less then 10% :- ", data_list_1[1])
print("Prediction With error less then 15% :- ", data_list_1[2])
print("Prediction With error less then 20% :- ", data_list_1[3])
print("Prediction With error less then 25% :- ", data_list_1[4])

Total Test Points 100
Prediction With error less then 5% :-  48
Prediction With error less then 10% :-  48
Prediction With error less then 15% :-  49
Prediction With error less then 20% :-  49
Prediction With error less then 25% :-  49


In [199]:
print(0,". ","        exact","      --      ","  predicted")
for i in range(len(x)):
    print(i+1,". ",exact[i],"  --  ",predicted[i])

0 .          exact       --         predicted
1 .  -324.7930739183425   --   186.93220000000002
2 .  -127.53640519351822   --   61.1888
3 .  -64.01612483854166   --   -59.10564000000001
4 .  -1176.0279286239718   --   -1174.8332
5 .  -1072.2312473986829   --   -1069.16794
6 .  -845.5359854194942   --   -845.2963199999999
7 .  -1225.1960533092158   --   -1224.1403
8 .  -272.8621104538813   --   153.75285999999997
9 .  -169.09268456400508   --   89.80212000000002
10 .  -211.45372164108812   --   113.6063
11 .  -50.28089345364566   --   -42.62512
12 .  -221.68492799337903   --   122.24818000000002
13 .  -1230.6592222825407   --   -1230.0961200000002
14 .  -395.10876186371786   --   -394.79438000000005
15 .  -97.75652599039209   --   43.69171999999999
16 .  -334.3031513381969   --   193.276
17 .  -68.97457353175726   --   25.024700000000003
18 .  -42.201532544552755   --   7.1480999999999995
19 .  -687.1485588800226   --   -686.95432
20 .  -212.91520957672185   --   115.89774000000003
21 .

# 1. 0I+5X+30Y+20Z

In [171]:
def mat_2(n):
    M_temp = n*I+(5+n)*X+(n+30)*Y+(n+20)*Z
    return(M_temp)

In [166]:
def y_point_2(x):
    y_temp = []
    a1 =     0.00185  
    b1 =   -0.004296  
    c1 =  -0.0003929  
    d1 =      0.1289 
    e =      0.8807 
    for i in x:
        temp = a1/(b1+c1*exp(-d1*i))+e
        y_temp.append(temp)
    return(y_temp)

In [169]:
def theta_point_2(x):
    z_temp = []
    a =      27.72 
    a2 =       -26.19
    b =    0.1275
    b2 =    0.1151
    c =       0.09268
    c2 =       0.09086
    e =      0.6415
    for i in x:
        temp = a/(1+b*exp(-c*i))+a2/(1+b2*exp(-c2*i))+d
        z_temp.append(temp)
    return(z_temp)

In [170]:
y2 = y_point_2(x)
t2 = theta_point_2(x)

In [184]:
data_list_2 = [0]*6
error_list_2 = []
exact_2 = []
predicted_2 = []
for i in range(len(x)):
    M = mat_2(x[i])
    results = la.eig(M)
    eigenval = min(results[0].real)
    
    
    
    qc= QuantumCircuit(1)
    qc.initialize(state(y[i],t[i]),[0])
    coeffs, obs_list = qml.utils.decompose_hamiltonian(M)
    H = qml.Hamiltonian(coeffs, obs_list)
    
    eigenval_predict = eigen_value_check(qc,H)
    
    
    error = (eigenval - eigenval_predict)/(eigenval)
    
    if error <0.05:
        data_list_2[0]+=1
    if 0.1>error :
        data_list_2[1]+=1
    if 0.15>error:
        data_list_2[2]+=1
    if 0.2>error :
        data_list_2[3]+=1
    if 0.25>error:
        data_list_2[4]+=1
    error_list.append(error)
    data_list_2[5]+=1
    
    exact_2.append(eigenval)
    predicted_2.append(eigenval_predict)
    

In [186]:
print("Total Test Points", data_list_1[5])
print("Prediction With error less then 5% :- ", data_list_2[0])
print("Prediction With error less then 10% :- ", data_list_2[1])
print("Prediction With error less then 15% :- ", data_list_2[2])
print("Prediction With error less then 20% :- ", data_list_2[3])
print("Prediction With error less then 25% :- ", data_list_2[4])

Total Test Points 100
Prediction With error less then 5% :-  44
Prediction With error less then 10% :-  46
Prediction With error less then 15% :-  48
Prediction With error less then 20% :-  48
Prediction With error less then 25% :-  48


In [201]:
print(0,". ","      exact","      --      ","  predicted")
for i in range(len(x)):
    print(i+1,". ",exact[i],"  --  ",predicted[i])

0 .        exact       --         predicted
1 .  -324.7930739183425   --   186.93220000000002
2 .  -127.53640519351822   --   61.1888
3 .  -64.01612483854166   --   -59.10564000000001
4 .  -1176.0279286239718   --   -1174.8332
5 .  -1072.2312473986829   --   -1069.16794
6 .  -845.5359854194942   --   -845.2963199999999
7 .  -1225.1960533092158   --   -1224.1403
8 .  -272.8621104538813   --   153.75285999999997
9 .  -169.09268456400508   --   89.80212000000002
10 .  -211.45372164108812   --   113.6063
11 .  -50.28089345364566   --   -42.62512
12 .  -221.68492799337903   --   122.24818000000002
13 .  -1230.6592222825407   --   -1230.0961200000002
14 .  -395.10876186371786   --   -394.79438000000005
15 .  -97.75652599039209   --   43.69171999999999
16 .  -334.3031513381969   --   193.276
17 .  -68.97457353175726   --   25.024700000000003
18 .  -42.201532544552755   --   7.1480999999999995
19 .  -687.1485588800226   --   -686.95432
20 .  -212.91520957672185   --   115.89774000000003
21 .  